In [5]:
import pandas as pd
import numpy as np
df=pd.read_csv("SMSSpamCollection.tsv",delimiter="\t",header=None)
df.columns=["label","body_text"]

,label,body_text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [13]:
import nltk
import string
import re
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
stopwords=stopwords.words("english")
ps=PorterStemmer()

In [27]:
def cleaning(text):
    text_nonpunc="".join([word for word in text if word not in string.punctuation])
    text_tokenize=re.split('\W+',text_nonpunc)
    text_stopwords=[ps.stem(word) for word in text_tokenize if word not in stopwords]
    return text_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
tfid_vec=TfidfVectorizer(analyzer=cleaning)
x_tfid=tfid_vec.fit_transform(df["body_text"])


In [51]:
x_tfid_df=pd.DataFrame(x_tfid.toarray())
x_tfid_df["body_len"]=df["body_text"].apply(lambda x: len(x)-len(" "))

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [62]:
x_train,x_test,y_train,y_test=train_test_split(x_tfid_df[:],df["label"])

In [65]:
def Forest(n_est,max_depth):
    rf=RandomForestClassifier(n_estimators=n_est,max_depth=max_depth,n_jobs=-1)
    model=rf.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    precision,recall,fscore,support=score(y_test,y_pred,pos_label="spam",average="binary")
    print("EST {} / DEPTH {} -------- precision {} / recall {} / accuracy {}".format(n_est,max_depth,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))
for n_est in [10,50,100,150,200]:
    for max_depth in [10,20,30,50,None]:
        Forest(n_est,max_depth)
    

EST 10 / DEPTH 10 -------- precision 1.0 / recall 0.288 / accuracy 0.895
EST 10 / DEPTH 20 -------- precision 1.0 / recall 0.59 / accuracy 0.94
EST 10 / DEPTH 30 -------- precision 1.0 / recall 0.663 / accuracy 0.95
EST 10 / DEPTH 50 -------- precision 0.988 / recall 0.834 / accuracy 0.974
EST 10 / DEPTH None -------- precision 0.994 / recall 0.81 / accuracy 0.971
EST 50 / DEPTH 10 -------- precision 1.0 / recall 0.244 / accuracy 0.889
EST 50 / DEPTH 20 -------- precision 1.0 / recall 0.61 / accuracy 0.943
EST 50 / DEPTH 30 -------- precision 1.0 / recall 0.673 / accuracy 0.952
EST 50 / DEPTH 50 -------- precision 1.0 / recall 0.741 / accuracy 0.962
EST 50 / DEPTH None -------- precision 1.0 / recall 0.829 / accuracy 0.975
EST 100 / DEPTH 10 -------- precision 1.0 / recall 0.215 / accuracy 0.884
EST 100 / DEPTH 20 -------- precision 1.0 / recall 0.6 / accuracy 0.941
EST 100 / DEPTH 30 -------- precision 1.0 / recall 0.717 / accuracy 0.958
EST 100 / DEPTH 50 -------- precision 1.0 / rec

In [68]:
rf_=RandomForestClassifier(n_estimators=50,max_depth=None,n_jobs=-1)
model=rf_.fit(x_train,y_train)
y_pred=model.predict(x_test)
precision,recall,fscore,support=score(y_test,y_pred,pos_label="spam",average="binary")
print( "precision {} / recall {} / accuracy {}".format(round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))

precision 1.0 / recall 0.805 / accuracy 0.971
